# GRT Integration Tests

In this Notebook we will test the integration of all components of GravRay in an actual physical test of functionalities.

In [1]:
from gravray import *
from gravray.util import *
from gravray.sampling import *
from gravray.spice import *
from gravray.orbit import *
from gravray.stats import *

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

Number of processors:  4


## NEOs Distribution 

In [ ]:
NEOs=MultiNormal()
weights=[0.6]
locs=[
    [0.5,0.5,-2.0],
    [2.0,0.3,-2.6]
]
scales=[
    [1.3,0.7,0.5],
    [0.4,0.9,1.6]
]
angles=[
    [-40.0*Angle.Deg,-86.0*Angle.Deg,0.0*Angle.Deg],
    [+80.0*Angle.Deg,-109.0*Angle.Deg,0.0*Angle.Deg]
]
NEOs.setUnflatten(weights,locs,scales,angles)

## Single ray

In [ ]:
#Preparation
Spice.loadKernels()

In [ ]:
#Parameters
body="EARTH"
lon=+61.1*Angle.Deg #deg
lat=+54.8*Angle.Deg #deg
alt=23.3*Const.km #m

In [ ]:
#Location
earth=Body(body)
chelyabinsk=Location(earth,lon,lat,alt)

In [ ]:
#Ray
A=101.1*Angle.Deg
h=15.9*Angle.Deg
v=-18.6*Const.kms
ray=GrtRay(chelyabinsk,A,h,v)

In [ ]:
#Time
tdb=Spice.str2t("02/15/2013 03:20:34 UTC")

In [ ]:
#Propagate ray
ray.updateRay(tdb)
ray.propagateRay()
Const.transformElements(ray.terminal.elements,[1/Const.au,Angle.Rad])

Jacobians:

p(Rimp) = p(Ehel) |dEhel/dehel| x |dehel/dXSoI| x |dXSoI/degeo| x |degeo/dXgeo| x |dXgeo/dRimp|

In [ ]:
# dXgeo/dRimp
ray.calcJacobiansBody()
dXgeodRimp=ray.Jcl
dXgeodRimp

In [ ]:
# degeo/dXgeo
geo_aelements=ray.conics[0]
geo_mu=geo_aelements[-1]
geo_elements=np.array(geo_aelements[:6])
geo=KeplerianOrbit(geo_mu)
geo.setElements(geo_elements,0)
geo.calcJacobians()
degeodXgeo=geo.Jck

#dXSoI/degeo
soi_aelements=ray.conics[1]
soi_mu=soi_aelements[-1]
soi_elements=np.array(soi_aelements[:6])
soi=KeplerianOrbit(soi_mu)
soi.setElements(soi_elements,0)
soi.calcJacobians()
dXSoIdegeo=geo.Jkc

#dehel/dXSoI
hel_aelements=ray.conics[2]
hel_mu=hel_aelements[-1]
hel_elements=np.array(hel_aelements[:6])
hel=KeplerianOrbit(hel_mu)
hel.setElements(hel_elements,0)
hel.calcJacobians()
deheldXSoI=hel.Jck

#dEhel/dehel
Const.transformElements(hel.elements,[1/Const.au,1.0],implicit=True)
hel.calcUelements()
hel.calcJacobiansMap()
dEheldehel=hel.JEe



In [ ]:
#Compute probability
pEhel=NEOs.pdf(hel.uelements[:3])
pEhel

In [ ]:
#|dEhel/dehel| x |dehel/dXSoI| x |dXSoI/degeo| x |degeo/dXgeo| x |dXgeo/dRimp|
pRimp=pEhel*np.abs(np.linalg.det(dEheldehel))*\
            np.abs(np.linalg.det(deheldXSoI))*\
            np.abs(np.linalg.det(dXSoIdegeo))*\
            np.abs(np.linalg.det(degeodXgeo))*\
            np.abs(np.linalg.det(dXgeodRimp))
pRimp